In [0]:
import keras
import random
import pickle
import numpy as np
import scipy.ndimage
import tensorflow as tf
import os
from PIL import Image
from keras import metrics
from random import shuffle
from keras.models import Model
from keras import backend as K
import matplotlib.pyplot as plt
from keras.optimizers import Adam, SGD
from keras.layers import  Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras import losses
from tiramisu_net import Tiramisu

In [0]:
model = Tiramisu(input_shape=(192,256,3),n_classes=1,attention=True)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
model.load_weights('../models/fcdn-isic-500-tl-gn.h5')

In [0]:
img='E:\\test\\images\\ISIC_0014931.jpg'
mask='E:\\test\\images\\ISIC_0014931_segmentation.png'

raw = Image.open(img)
gmsk = Image.open(mask)

height = 192
width = 256

dim = (width, height)


gmsk = np.array(gmsk.resize(dim))/255.
raw = np.array(raw.resize(dim))/255.

#predict the mask 
pred = model.predict(np.expand_dims(raw, 0))

#mask post-processing 
msk  = pred.squeeze()
msk = np.stack((msk,)*3, axis=-1)
gmsk  = gmsk.squeeze()
gmsk = np.stack((gmsk,)*3, axis=-1)

msk[msk >= 0.5] = 1 
msk[msk < 0.5] = 0 

#show the mask and the segmented image 
combined = np.concatenate([raw,msk,gmsk], axis = 1)
plt.imshow(combined)

#plt.savefig('test_result.png')